### Using SQL

This notebook shows several examples how to select data from the database using just SQL.

Using some PostgreSQL UI like [DBeaver](https://dbeaver.io/), we can inspect all available data. 
It is an ideal form for exploring the data in details.
Mainly, we can query whatever data we want. Then, we can order it by any column(s) up and down when trying to find extremes. We can modify SQL formula or selection easily. We can perform various calculations, filter results in UI and so on.

## Examples

**Which strategies do trade Facebook?**

~~~sql
select systemid, systemname, Date(to_timestamp(added)) as Added , startingcash 
from c2systems
where systemid in (select distinct(systemid) from c2ex_trades where symbol = 'FB');
~~~

![Image](docs/WhoTradesFacebook.png)
****


**Show "extreme-os" details.**
~~~sql
select systemname, to_timestamp(added), startingcash, openpositionspl, closedpl, cash 
from c2systems 
where systemid = 13202557;
~~~

![Image](./docs/Extremeos_details.png)

****


**Show "extreme-os" results.**
~~~sql
select * 
from c2ex_results 
where systemid = 13202557;
~~~

![Image](./docs/Extremeos_results.png)

****


**Show trading signals of Facebook in the "extreme-os" strategy.**

~~~sql
select tradedwhen, action, quant, tradeprice 
from c2ex_signals 
where systemid = 13202557 and symbol = 'FB' order by tradedwhen;
~~~

![Image](./docs/FacebookSignals.png)

****


**Closed trades results.**

Facebook trades made by the **extreme-os** strategy.

~~~sql
select entrytime , exittime, closedquant, "result"  
from c2ex_trades cet 
where systemid = 13202557 and symbol = 'FB';
~~~

![Image](./docs/Extremeos_trades.png)

****


**Show history of "Annual Return" in the "extreme-os" strategy.**
~~~sql
select date, statval 
from historical_stats_modern 
where systemid = 13202557 and statname = 'cARdefault' order by 1;
~~~

![Image](./docs/Extremeos_AnnReturn.png)

****


**Show a history of several statistics of the "extreme-os" strategy.**

~~~sql
select 
base.yyyymmdd,  
cARdefault.statval as AnnReturn, 
profitFactor.statval as ProfitFactor,
maxdrawdownPcnt.statval as MaxDDPcnt,
numtrades.statval as NumTrades,
avgDollarPosWinners.statval as AvgWin,
avgDollarPosLosers.statval as AvgLos,
avgDollarPosWinners.statval / avgDollarPosLosers.statval as WinRatio
-- Let's get systemAgeDays as a base data source for Date and SystemId
from (select date, systemid from historical_stats_modern where statname = 'systemAgeDays') base
join historical_stats_modern cARdefault on cARdefault.date = base.date and cARdefault.systemid = base.systemid  and cARdefault.statname = 'cARdefault'
join historical_stats_modern maxdrawdownPcnt on maxdrawdownPcnt.date = base.date and maxdrawdownPcnt.systemid = base.systemid  and maxdrawdownPcnt.statname = 'maxdrawdownPcnt'
join historical_stats_modern profitFactor on profitFactor.date = base.date and profitFactor.systemid = base.systemid and profitFactor.statname = 'profitFactor'
join historical_stats_modern numtrades on numtrades.date = base.date and numtrades.systemid = base.systemid and numtrades.statname = 'numtrades'
join historical_stats_modern avgDollarPosLosers on avgDollarPosLosers.date = base.date and avgDollarPosLosers.systemid = base.systemid and avgDollarPosLosers.statname = 'avgDollarPosLosers'
join historical_stats_modern avgDollarPosWinners on avgDollarPosWinners.date = base.date and avgDollarPosWinners.systemid = base.systemid and avgDollarPosWinners.statname = 'avgDollarPosWinners'
where base.systemid = 13202557 
order by 1;
~~~
 
![Image](./docs/Extremeos_stats.png)

****


### Daniil's formula (v1)

Replicate Collective2 Scoring Workbench functionality using just SQL.

This is an SQL code for [Daniil's formula (v1)](https://svancara.github.io/C2QuantAPI/daniil_formula_v1.html). 


~~~sql
#  ---------- Plain with all data  ----------
select * from (
			select
			    StrategyId, 
			    StrategyName, 
			    Added,
			    -- Risk
			    ((20 - MaxDrawdownPcnt * (20/45))
			    + case when MaxWorstLossPercentEquity060 is null 
                       then 0 else 20 - MaxWorstLossPercentEquity060 * (20.0/8) end  
			    + (10 - OptionPercent * (10/1))
			    + case when ShortOptionsCovered >= 100 then 10 else ShortOptionsCovered / 100 * 10 end 
			    -- Behavior
			    + case when AgeDays >= 720 then 40 else AgeDays/720 * 40 end  
			    + (20 - DailyMaxLevMax * (20/10)) 
			    + case when TradeDaysAgo <= 60 then 20 else 0 end
			    + case when PcntMonthsProfitable >= 100 then 5 else PcntMonthsProfitable / 100 * 5 end
			    + case when NumTrades >= 100 then 20 else NumTrades / 100 * 20 end
			    + case when DeltaEquity90Days <= 0 and DeltaEquity90Days > -7 then 5 else 0 end
			    -- Profitability 
			    + case when AnnReturn > 0.6 then 20 else AnnReturn / 0.6 * 20 end)
			    as Score, 
			    c2score_ScoringWorkbenchView.*
			from c2score_ScoringWorkbenchView
			where AgeDays > 90 and AnnReturn > 0 and DeltaEquity180Days > 0
			) as sub
 where Score > 0
 order by Score desc;
~~~

![Image](./docs/PostgreSQL_Daniil01_plain.png)